### Sobre este notebook
Notebook para ler o sample e acrescentar uma coluna de data de indexação falsa em diferente períodos, que será usado pelo script de notificação de novos documentos

In [1]:
import pandas as pd

In [2]:
original_sample_files = [
    '../indexer/indices-sample/diarios_segmentado/diarios-amm-segmentado.csv',
    '../indexer/indices-sample/diarios/diarios-amm.csv',
    '../indexer/indices-sample/diarios/diarios-bh.csv',
    '../indexer/indices-sample/licitacoes/licitacoes-obras.csv',
    '../indexer/indices-sample/processos/processos-trf.csv'
]

In [7]:
for i, filepath in enumerate(original_sample_files):
    filepath = original_sample_files[i]
    df = pd.read_csv(filepath)
    
    # no caso do diário segmentado tenho que fazer diferente
    # pois os segmentos com o mesmo pai devem possuir a mesma data
    if i == 0:
        # seta a data do documento de acordo com o ID do pai
        id_pais = df['id_pai'].unique()
        data_pais = {}
        block_size = int(len(id_pais) / 5)
        for i in range(5):
            start = block_size * i
            end = block_size * i + block_size
            for j in range(start, end):
                data_pais[id_pais[j]] = '2021-0'+str(i+1)+'-01'

        # percorre o DF atualizando as datas pelo id_pai
        df['data_indexacao'] = ''
        for id_pai, data_indexacao in data_pais.items():
            df.loc[df['id_pai']==id_pai, 'data_indexacao'] = data_indexacao
    # ----------------------------------------------------------------------
    else:
        # cria uma coluna vazia pra todo o DF e depois preenche bloco a bloco
        # com a data avançando um mês a cada bloco
        df['data_indexacao'] = ''
        block_size = int(len(df) / 5)
        for i in range(5):
            start = block_size * i
            end = block_size * i + block_size
            df.loc[start:end, 'data_indexacao'] = '2021-0'+str(i+1)+'-01'

    # -----------------------------------------------------------------------
    # salva em um novo CSV com mesmo nome + o sufixo '_novo'
    path_name, file_name = filepath.rsplit('/', 1)
    file_name = file_name.split('.')[0]
    df.to_csv(path_name+'/'+file_name+'_novo.csv', index=False)